In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


def plot_metric(
    df_default,
    df_harmonic,
    metric,
    ylabel,
    title,
    constant_default=None,
    constant_harmonic=None,
):
    default_colour = "C0"
    harmonic_colour = "C1"

    plt.figure(figsize=(10, 6))

    plt.plot(
        df_default.index,
        df_default[metric],
        marker="o",
        label="Default",
        linewidth=2,
        markersize=1,
        linestyle="-",
        color=default_colour,
    )
    plt.plot(
        df_harmonic.index,
        df_harmonic[metric],
        marker="s",
        label="Harmonic (ß=1)",
        linewidth=2,
        markersize=1,
        linestyle=":",
        color=harmonic_colour,
    )

    if constant_default is not None:
        plt.axhline(
            y=constant_default,
            linestyle="--",
            linewidth=1.5,
            label="Default baseline",
            color=default_colour,
        )
    if constant_harmonic is not None:
        plt.axhline(
            y=constant_harmonic,
            linestyle="--",
            linewidth=1.5,
            label="Harmonic baseline",
            color=harmonic_colour,
        )

    plt.xlabel("|CB|")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# GTD

df_default = pd.read_csv("results/Q_gtd_default.csv", sep=";", index_col="|Q|")
df_harmonic = pd.read_csv("results/Q_gtd_harmonic_1.csv", sep=";", index_col="|Q|")
plot_metric(
    df_default,
    df_harmonic,
    "mu",
    "$\mu$",
    "Mean Number of Best Precedents",
    constant_default=20.49,
    constant_harmonic=26.17,
)
plot_metric(
    df_default,
    df_harmonic,
    "mu_n",
    "$\mu_n$",
    "Mean Best Precedents (Non-Trivial)",
    constant_default=21.56,
    constant_harmonic=29.46,
)
plot_metric(
    df_default,
    df_harmonic,
    "Ninc",
    "$N_{inc}$",
    "Inconsistent Forcings",
    constant_default=4906.67,
    constant_harmonic=0,
)
plot_metric(
    df_default,
    df_harmonic,
    "Ndel",
    "$N_{del}$",
    "Deletions Required",
    constant_default=240.33,
    constant_harmonic=0,
)
plot_metric(df_default, df_harmonic, "Ntws", "$N_{tws}$", "Trivial Winning Strategies")
plot_metric(df_default, df_harmonic, "Nn", "$N_n$", "Non-Trivial Winning Strategies")

In [ ]:
# Churn

df_default = pd.read_csv("results/Q_gtd_default.csv", sep=";", index_col="|Q|")
df_harmonic = pd.read_csv("results/Q_gtd_harmonic_1.csv", sep=";", index_col="|Q|")
plot_metric(
    df_default,
    df_harmonic,
    "mu",
    "$\mu$",
    "Mean Number of Best Precedents",
    constant_default=20.49,
    constant_harmonic=26.17,
)
plot_metric(
    df_default,
    df_harmonic,
    "mu_n",
    "$\mu_n$",
    "Mean Best Precedents (Non-Trivial)",
    constant_default=21.56,
    constant_harmonic=29.46,
)
plot_metric(
    df_default,
    df_harmonic,
    "Ninc",
    "$N_{inc}$",
    "Inconsistent Forcings",
    constant_default=4906.67,
    constant_harmonic=0,
)
plot_metric(
    df_default,
    df_harmonic,
    "Ndel",
    "$N_{del}$",
    "Deletions Required",
    constant_default=240.33,
    constant_harmonic=0,
)
plot_metric(df_default, df_harmonic, "Ntws", "$N_{tws}$", "Trivial Winning Strategies")
plot_metric(df_default, df_harmonic, "Nn", "$N_n$", "Non-Trivial Winning Strategies")